# Python 関数のデプロイ
Deployable Functionの使い方

このノートブックでは、Python 関数を Watson Machine Learning サービスにデプロイする方法を示します。

bash の知識があると便利です。このノートブックでは、Python 3.10 を使用します。

## 学習目標

このノートブックの学習目標は次のとおりです。

- Watson Machine Learning インスタンスの操作
- Python 関数のオンライン デプロイ
- デプロイされた関数を使用したデータのスコアリング

## 内容

このノートブックには、次の部分が含まれています。

1. [セットアップ](#setup)
2. [関数の作成](#create)
3. [関数のアップロード](#upload)
4. [Web サービスの作成](#deploy)
5. [スコアリング](#score)
6. [クリーンアップ](#cleanup)
7. [要約と次のステップ](#summary)

### 関数のデプロイ - Docs | IBM Cloud Pak for Data as a Service 
https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-deploy-functions.html?locale=ja&context=cpdaas

### Watson Machine Learning での Python 関数のデプロイ - Docs | IBM Cloud Pak for Data as a Service 
https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-deploy-py-function.html?context=cpdaas&context=cpdaas&audience=wdp&locale=ja


<a id="setup"></a>
## 1. 環境を設定する

このノートブックのサンプル コードを使用する前に、次の設定タスクを実行する必要があります:

- Cloud Pack for Data 管理者に連絡して、API KEYとエンドポイントURLを尋ねます


### `ibm_watsonx_ai` パッケージをインストールしてインポートします
**注:** `ibm-watsonx-ai` のドキュメントは <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">こちら</a> にあります。

In [1]:
#!pip install -U ibm-watsonx-ai
#py -3.10 -m pip install ibm-watsonx-ai


In [2]:
# ibm_watsonx_aiのバージョン確認
import ibm_watsonx_ai


print(ibm_watsonx_ai.__version__ )

1.1.2


### watsonx.ai  への接続

IBM Cloud Pack for Data で watsonx.ai  サービスを認証します。プラットフォームの `url`、`api_key` を指定する必要があります。<br>
注: api_key を確認するには、IBM Cloud コンソールの API キーを参照してください。<br>
https://cloud.ibm.com/iam/apikeys<br>
End point<br>
https://ibm.github.io/watsonx-ai-python-sdk/setup_cloud.html

In [3]:
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

api_key = 'xxxxxxxxxxxxx' #240816itz13
# tokyo Public endpoint
url = 'https://jp-tok.ml.cloud.ibm.com'

In [4]:
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials

#APIキーを使用してクライアントを作成する
credentials = Credentials(
    api_key = api_key,
    url= url
)

client = APIClient(credentials)

### デプロイメント・スペースの操作

まず、作業に使用するデプロイメント・スペースを作成する必要があります。スペースがまだ作成されていない場合は、`{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` を使用して作成できます。

- [新しいデプロイメント スペース] をクリックします
- 空のスペースを作成します
- スペースの `Settings` タブに移動します
- `space_id` をコピーして下に貼り付けます

**ヒント**: 作業用のスペースを準備するために SDK を使用することもできます。詳細については、[こちら](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb) を参照してください。

**アクション**: 以下にスペース ID を割り当てます

In [5]:
space_id = 'PASTE YOUR SPACE ID HERE'
space_id = 'xxxxxxxxxxxxx' #240816dps_itz13


`list` メソッドを使用してスペースを確認します。

In [6]:
client.spaces.list().query('ID == @space_id')

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50


,ID,NAME,CREATED
0,1f3faf60-b123-400e-a03a-8caf09586ca4,240816dps_itz13,2024-08-16T05:59:39.409Z


ibm_watsonx_ai で利用可能なすべてのリソースと対話できるようにするには、使用する **スペース** を設定する必要があります。

In [7]:
#client.set.default_space(space_id)
client_space = APIClient(credentials, space_id = space_id)

<a id="create"></a>
## 2. 関数の作成

「score」という名前のネストされた関数を使用して Python クロージャーを作成します。

#### 関数を定義する
- 名前は「score」でなければなりません
- score関数の引数
    - 単一の JSON 入力パラメーターを受け入れる必要があります
    - input_dataのキーを持つ必要があります。
    - 入力パラメーターには、以下の「values」という名前の配列が含まれている必要があります。
    - 入力パラメーターには、追加の名前と値のペアを含めることもできます
- score関数の戻り値
    - score 関数が JSON 直列化可能オブジェクトを返す必要があります (例: ディクショナリーまたはリスト)
    - predictionsのキーを持つ必要があります。

In [8]:
def my_deployable_function():
    """
    Deployable python function with score
    function implemented.
    """
        
    def score(payload):
        """
        Score method.
        """
        return {'predictions': [{'values': [payload['input_data'][0]["values"]]}]}
       
    return score

#### ローカルで呼び出し可能オブジェクトをテストする


In [9]:
data = 'hello world'
func_result = my_deployable_function()({
    "input_data": [{
        "values" : data
    }]
})

print(func_result["predictions"][0]["values"])

['hello world']


<a id="upload"></a>
## 3. Python 関数のアップロード

このセクションでは、Python 関数をDeployment Spaeにアップロードする方法を学びます。

Baseソフトウェア仕様を選択します。

In [10]:
import pandas as pd
df_swspec=client_space.software_specifications.list(limit=300)
pd.set_option('display.max_rows', 300)
df_swspec.sort_values('NAME')

,NAME,ID,TYPE,STATE,REPLACEMENT
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,retired,runtime-23.1-py3.10
106,autoai-kb_3.0-py3.6,d139f196-e04b-5d8b-9140-9a10ca1fa91a,base,retired,autoai-kb_rt23.1-py3.10
60,autoai-kb_3.1-py3.7,632d4b22-10aa-5180-88f0-f52dfb6444d7,base,retired,autoai-kb_rt23.1-py3.10
90,autoai-kb_3.3-py3.7,a545cca3-02df-5c61-9e88-998b09dc79af,base,retired,autoai-kb_rt23.1-py3.10
108,autoai-kb_3.4-py3.8,da9b39c3-758c-5a4f-9cfd-457dd4d8c395,base,retired,autoai-kb_rt23.1-py3.10
111,autoai-kb_rt22.1-py3.9,db6afe93-665f-5910-b117-d879897404d9,base,retired,autoai-kb_rt23.1-py3.10
14,autoai-kb_rt22.2-py3.10,125b6d9a-5b1f-5e8d-972a-b251688ccf40,base,constricted,autoai-kb_rt23.1-py3.10
98,autoai-kb_rt23.1-py3.10,b4246d87-d8b3-5621-9fea-76a8ea452616,base,supported,autoai-kb_rt24.1-py3.11
73,autoai-kb_rt24.1-py3.11,82f1bc6d-59c3-55e8-b612-cd908a2f7cc7,base,supported,
56,autoai-obm_2.0,5c2e37fa-80b8-5e77-840f-d912469614ee,base,retired,autoai-obm_3.2


In [11]:
# ソフトウェア仕様のIDを取得
sw_spec_uid = client.software_specifications.get_uid_by_name("runtime-23.1-py3.10")

#### 関数を保存する

In [12]:
meta_props = {
    client.repository.FunctionMetaNames.NAME: "deployable function test",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

function_details = client_space.repository.store_function(meta_props=meta_props, function=my_deployable_function)
function_id = client_space.repository.get_function_id(function_details)

#### 関数の詳細を取得する

In [13]:
client_space.repository.get_details(function_id)

{'entity': {'software_spec': {'id': '336b29df-e0e1-5e7d-b6a5-f6ab722625b2',
   'name': 'runtime-23.1-py3.10'},
  'type': 'python'},
 'metadata': {'created_at': '2024-08-16T08:51:39.721Z',
  'id': '34498409-d80a-4b67-bd0a-f33aa1003241',
  'modified_at': '2024-08-16T08:51:41.259Z',
  'name': 'deployable function test',
  'owner': 'IBMid-120000E7EC',
  'space_id': '1f3faf60-b123-400e-a03a-8caf09586ca4'},
 'system': {'warnings': []}}

**注:** 関数が Watson Machine Learning Service に正常に保存されたことがわかります。

In [14]:
#client.repository.delete(function_uid)
client_space.repository.list_functions()

,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,34498409-d80a-4b67-bd0a-f33aa1003241,deployable function test,2024-08-16T08:51:39.002Z,python,supported,
1,8009763f-1b03-49e7-a18b-8c6c77e8ba21,deployable function test,2024-08-16T08:40:01.002Z,python,supported,
2,69ea58ed-b76c-40ab-b098-73a19c663b29,deployable function test,2024-08-16T07:10:57.002Z,python,supported,


<a id="deploy"></a>
## 4. オンライン デプロイメントを作成する
以下のコマンドを使用して、ストアド ファンクション (Web サービス) のオンライン デプロイメントを作成できます。

#### Python関数のオンラインデプロイメントを作成する

In [15]:
metadata = {
    client_space.deployments.ConfigurationMetaNames.NAME: "Deployment of function",
    client_space.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment_details  = client_space.deployments.create(function_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '34498409-d80a-4b67-bd0a-f33aa1003241' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='0ab7ab46-6429-454a-8d1e-bfc5af9e2a96'
-----------------------------------------------------------------------------------------------




In [16]:
client_space.deployments.list()

,ID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,0ab7ab46-6429-454a-8d1e-bfc5af9e2a96,Deployment of function,ready,2024-08-16T08:51:44.897Z,function,supported,
1,ac890519-3f76-4c15-aca7-3c7850fc976b,Deployment of function,ready,2024-08-16T08:40:09.919Z,function,supported,
2,5db0d05f-b92f-408c-a16c-a830d7359210,Deployment of function,ready,2024-08-16T07:37:35.799Z,function,supported,


デプロイメント ID を取得します。

In [17]:
deployment_id = client.deployments.get_id(function_deployment_details)
print(deployment_id)

0ab7ab46-6429-454a-8d1e-bfc5af9e2a96


<a id="score"></a>
## 5. スコアリング

`score` メソッドを使用して、スコアリング レコードを Web サービス デプロイメントに送信できます。

In [18]:
payload = {
    "input_data": [{
        'values': 'hello again'
    }]
}

predictions = client_space.deployments.score(deployment_id, payload)
#print(predictions['predictions'][0]["values"])
print(predictions)

{'predictions': [{'values': ['hello again']}]}


<a id="cleanup"></a>
## 6. クリーンアップ

作成されたすべてのアセットをクリーンアップする場合は、次の手順に従ってください:
- デプロイメントの削除
- 関数の削除

このサンプルに従ってください [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

In [19]:
client_space.deployments.delete(deployment_id)

'SUCCESS'

In [20]:
client_space.repository.delete(function_id)


'SUCCESS'

<a id="summary"></a>
## 7. まとめと次のステップ

このノートブックは正常に完了しました。関数のデプロイメントとスコアリングを行うために Watson Machine Learning を使用する方法を学習しました。
その他のサンプル、チュートリアル、ドキュメント、ハウツー、ブログ投稿については、[オンライン ドキュメント](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics) をご覧ください。

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.